<a href="https://colab.research.google.com/github/PRAN20/LeafSnap-Model_1/blob/main/alexnet_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


loading model

In [2]:
import keras
model= keras.models.load_model("/content/drive/MyDrive/LeafSnap-1.hdf5")

viewing model layers

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                             
                                                        

selecting model layers from 0 to n-1 and initializing it to new_model / tried pop function but wasn't working

In [4]:
import tensorflow as tf
new_model = tf.keras.models.Sequential(model.layers[:-1])

checking new_model layers

In [5]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                             
                                                        

getting dataset

In [6]:
!unzip "/content/drive/MyDrive/LeafSnap.zip" -d "DataSet"

Streaming output truncated to the last 5000 lines.
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-03-1.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-03-2.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-03-3.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-03-4.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-04-1.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-04-2.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-04-3.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-04-4.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-05-1.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-05-2.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-05-3.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-05-4.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-06-1.jpg  
  inflating: DataSet/images/lab/quercus_coccinea/wb1472-06-2.jpg  
  inflating

checking if prediction is giving right number of features

In [7]:
import cv2
import numpy as np
img1= cv2.imread("/content/DataSet/images/field/abies_nordmanniana/13291651121149.jpg",1)
img1 = cv2.resize(img1,(224,224))
img1= np.expand_dims(img1,axis=0)
(new_model.predict(img1/255))[0].flatten().shape

(1000,)

In [9]:
!pip install split-folders

In [10]:
import splitfolders

splitfolders.ratio("/content/DataSet/images/field", output="/content/Images",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Copying files: 7719 files [00:01, 4254.90 files/s]


passing all images from all folders in train and storing predicted features by new_model into train_imgs array / storing folder name as label in train_labels array.

In [14]:
import os
import cv2
import numpy as np
train_imgs=[]
train_labels=[]
n=1
for folder in os.listdir("/content/Images/train"):
  for imgs in os.listdir("/content/Images/train/"+folder):
    img1_path = cv2.imread("/content/Images/train/"+folder+"/"+imgs,1)
    img1 = cv2.resize(img1_path,(224,224))
    img1= np.expand_dims(img1,axis=0)
    img1_embedding = new_model.predict(img1/255)[0].flatten()
    train_imgs.append(img1_embedding)
    train_labels.append(folder)

same thing with test folder

In [16]:
import os
import cv2
import numpy as np
test_imgs=[]
test_labels=[]
n=1
for folder in os.listdir("/content/Images/test"):
  for imgs in os.listdir("/content/Images/test/"+folder):
    img1_path = cv2.imread("/content/Images/test/"+folder+"/"+imgs,1)
    img1 = cv2.resize(img1_path,(224,224))
    img1= np.expand_dims(img1,axis=0)
    img1_embedding = new_model.predict(img1/255)[0].flatten()
    test_imgs.append(img1_embedding)
    test_labels.append(folder)

converting to numpy array

In [17]:
train_imgs= np.array(train_imgs)
train_labels= np.array(train_labels)

checking if we got correct array as planned

In [18]:
train_imgs.shape, train_labels.shape

((6109, 1000), (6109,))

saving train_imgs to X_train.csv

In [20]:
cd /content

/content


In [21]:
import numpy
a = numpy.asarray(train_imgs)
numpy.savetxt("X_train.csv", a, delimiter=",")

saving train_labels to y_train.csv

In [25]:
a = numpy.asarray(train_labels)
numpy.savetxt("y_train.csv", a,  fmt='%s',delimiter=",")

converting test images and labels to numpy array

In [26]:
test_imgs= np.array(test_imgs)
test_labels= np.array(test_labels)
test_imgs.shape, test_labels.shape

((917, 1000), (917,))

saving test_imgs to X_test and test_labels to y_test.csv

In [28]:
a = numpy.asarray(test_imgs)
numpy.savetxt("X_test.csv", a, delimiter=",")
a = numpy.asarray(test_labels)
numpy.savetxt("y_test.csv", a, delimiter=",",fmt='%s')

In [29]:
from zipfile import ZipFile
import os
  
def get_all_file_paths(directory):
    file_paths = []
  
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    return file_paths        
  
def main():
    directory = '/content/LeafSnap-1 Csv'
    file_paths = get_all_file_paths(directory)
    print('Following files will be zipped:')
    for file_name in file_paths:
        print(file_name)
  
    with ZipFile('my_python_files.zip','w') as zip:
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
  
if __name__ == "__main__":
    main()

Following files will be zipped:
/content/LeafSnap-1 Csv/X_test.csv
/content/LeafSnap-1 Csv/y_train.csv
/content/LeafSnap-1 Csv/y_test.csv
/content/LeafSnap-1 Csv/X_train.csv
All files zipped successfully!
